In [1]:
import numpy as np # We'll be storing our data as numpy arrays
import os # For handling directories
from PIL import Image # For handling the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting
lookup = dict()
reverselookup = dict()
count = 0
#print(os.listdir('../input/leapgestrecog/leapGestRecog/01/'))
for j in os.listdir('./output/'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

{'test': 0, 'train': 1, 'val': 2}

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_rows, img_cols = 120 , 320
num_classes = 3

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
eval_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './output/train/',
        target_size=(img_rows, img_cols),
        batch_size= 32,
        color_mode = 'grayscale',
        class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(
        './output/val/',
        target_size=(img_rows, img_cols),
        batch_size= 32,
        color_mode = 'grayscale',
        class_mode='categorical')
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', strides = (2,2)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', strides = (2,2)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])


Found 2400 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [4]:
model.fit_generator(
        train_generator,
        validation_data=valid_generator,
        epochs=2)

Epoch 1/2
75/75 [==============================] - 115s 2s/step - loss: 0.7796 - acc: 0.6200 - val_loss: 0.2896 - val_acc: 0.9233
Epoch 2/2
75/75 [==============================] - 110s 1s/step - loss: 0.1879 - acc: 0.9367 - val_loss: 0.0526 - val_acc: 0.9867


In [5]:
model.save("Leap_motion_model.h5")

In [6]:
eval_generator = eval_datagen.flow_from_directory(
        './output/test/',
        target_size=(img_rows, img_cols),
        color_mode = 'grayscale')
        #batch_size=32,
        #class_mode='categorical')

Found 300 images belonging to 3 classes.


In [7]:
[loss,score] = model.evaluate_generator(eval_generator)
print("Loss: "+ str(loss) +" Accuracy: " + str(score))

Loss: 0.044668314407269157 Accuracy: 0.99
